## Sections:

1.   Load and prepare data
2.   Data preprocessing

    *   Filter and noise reduction
    *   Extract motion sequence
    *   Rotate data
    *   Segment trial into samples
    *   Resampling and normalization
    *   Remove malicious steps
3.   Neural Network
    
    *   Data set-up
    *   Training and evaluation




# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random 
from os import listdir
from os.path import isfile, join
import glob
from pathlib import Path
from scipy.signal import butter, sosfilt
import scipy as sp
from scipy import signal
from scipy.spatial.transform import Rotation as R
import math
import tensorflow as tf

# 1. Load and prepare data

Define directory of Smartphone data.

The folder should contain:
* The subfolders of all experiments

The parent folder should contain 
* anthro.xlsx
  


In [ ]:
input_dir = 'C:/Users/Mridu/Desktop/CIiE/Project/test/'
output_dir = '' 
input_anthropology_file = ''

In [ ]:
def get_frequency(data): 
    # gets the frequency of the accelerometer/gyroscope data and returns it as a dataframe
    time = data.iloc[:]['Time [s]']
    time = np.array(pd.DataFrame(time))     

    time_duration = time.max() - time.min()
    number_datapoints = time.size
    
    frequency = number_datapoints/time_duration
    
    frequency_df = pd.DataFrame(data = ([frequency]), columns = ['frequency [Hz]'])
    
    return frequency_df

In [ ]:
def unify_anthropometric_data(anthropometric_data = pd.DataFrame()):        
    # trys to unify the manual written anthropology data
    # i used the data as string to easyly change the commas to points and to cut out any possible letters (e.g.: kg)
    
    sex = anthropometric_data.iloc[0]['subject_sex']
    sex_str = str(sex)
    height = anthropometric_data.iloc[0]['subject_height [m]']
    height_str = str(height)                                                
    height_str = height_str.replace(',','.')
    weight = anthropometric_data.iloc[0]['subject_weight [kg]']
    weight_str = str(weight)
    weight_str = weight_str.replace(',','.')
    age = anthropometric_data.iloc[0]['subject_age [y]']
    age_str = str(age)
    
    if ('f' in sex_str) or ('w' in sex_str) or ('F' in sex_str) or ('W' in sex_str):                 # checks for 'F/female' and 'W/woman'
        
        anthropometric_data.at[0,'subject_sex'] = 0     # 0 = female
        
    else: 
        
        anthropometric_data.at[0,'subject_sex'] = 1     # 1 = male
        
    for i in range(len(height_str)):
        
        if not (height_str[i].isdigit() or (height_str[i] == '.')):
            
            height_str = height_str[0:i]                # cuts of the string after the numbers end ('1.85 m' -> '1.85')
            
            break
     
    if not (len(height_str) == 0):    
                
        if float(height_str) > 100:
            
            height_str = float(height_str)/100              # makes sure the unit is [m] and not [cm]
        
    for i in range(len(weight_str)):
        
        if not (weight_str[i].isdigit() or (weight_str[i] == '.')):
            
            weight_str = weight_str[0:i]                # cuts of the string after the numbers end ('65 kg' -> '65')
            
            break    
        
    for i in range(len(age_str)):
        
        if not age_str[i].isdigit():
            
            age_str = age_str[0:i]                      # cuts of the string after the numbers end ('25 y' -> '25')
            
            break
    
    if height_str and weight_str and age_str:
        
        anthropometric_data.at[0,'subject_height [m]'] = float(height_str)  # updating the data
        anthropometric_data.at[0,'subject_weight [kg]'] = float(weight_str) # updating the data
        anthropometric_data.at[0,'subject_age [y]'] = float(age_str)        # updating the data
        
        return anthropometric_data
    
    else:
    
        return pd.DataFrame()

In [ ]:
def unify_measurement_label(data_in = pd.DataFrame(), data_name = ''):
    # trys to unify the different sensor names/labels of the different smartphones
    
    data_out = data_in
    
    for column in data_in.columns:
        
        if ('Time' in column) or ('time' in column):
            
            data_out = data_out.rename(columns = {column: 'Time [s]'})
        
        elif ('acc' in column) or ('Acc' in column) or ('m/s^2' in column):
            
            if ('x' in column) or ('X' in column):
                
                data_out = data_out.rename(columns = {column: 'acceleration_x [m/s^2]'})

            elif ('y' in column) or ('Y' in column):
                
                data_out = data_out.rename(columns = {column: 'acceleration_y [m/s^2]'})
                
            elif ('z' in column) or ('Z' in column):
                
                data_out = data_out.rename(columns = {column: 'acceleration_z [m/s^2]'})
                
        elif ('gyroscope' in column) or ('Gyroscope' in column) or ('rad' in column):
            
            if ('x' in column) or ('X' in column):
                
                data_out = data_out.rename(columns = {column: 'angular_velocity_x [rad/s]'})

            elif ((column.count('y') == 2) and ('gyroscope' in column)) or ((column.count('y') == 2) and ('Gyroscope' in column)) or (
                    ('y' in column) and ('rad' in column) and (not(('gyroscope' in column) or ('Gyroscope' in column)))) or (
                    ('Y' in column) and ('rad' in column) and (not(('gyroscope' in column) or ('Gyroscope' in column)))
                    ) or ((column.count('Y') == 2) and ('gyroscope' in column)) or ((column.count('Y') == 2) and ('Gyroscope' in column)):
                
                data_out = data_out.rename(columns = {column: 'angular_velocity_y [rad/s]'})
                
            elif ('z' in column) or ('Z' in column):
                
                data_out = data_out.rename(columns = {column: 'angular_velocity_z [rad/s]'})
            
    if not (('Time [s]' in data_out.columns) and (('acceleration_x [m/s^2]' in data_out.columns) or ('angular_velocity_x [rad/s]' in data_out.columns)
    ) and (('acceleration_y [m/s^2]' in data_out.columns) or ('angular_velocity_y [rad/s]' in data_out.columns)) and (
             ('acceleration_z [m/s^2]' in data_out.columns) or ('angular_velocity_z [rad/s]' in data_out.columns)) and (data_out.columns.size == 4)):
        
        if not data_name:
            
            return pd.DataFrame()
    
        else:
            
            if 'Acc' in  data_name:
                
                if data_in.columns.size == 5:
        
                    data_out = pd.DataFrame(data = np.array(data_in)[:,1:], columns = ['Time [s]', 'acceleration_x [m/s^2]', 'acceleration_y [m/s^2]', 'acceleration_z [m/s^2]'])
                    
                else:
                
                    data_out = pd.DataFrame(data = np.array(data_in), columns = ['Time [s]', 'acceleration_x [m/s^2]', 'acceleration_y [m/s^2]', 'acceleration_z [m/s^2]'])
                
                
                
            elif 'Gyr' in data_name:
                
                if data_in.columns.size == 5:
        
                    data_out = pd.DataFrame(data = np.array(data_in)[:,1:], columns = ['Time [s]', 'angular_velocity_x [rad/s]', 'angular_velocity_y [rad/s]', 'angular_velocity_z [rad/s]'])
                
                else:
                
                    data_out = pd.DataFrame(data = np.array(data_in), columns = ['Time [s]', 'angular_velocity_x [rad/s]', 'angular_velocity_y [rad/s]', 'angular_velocity_z [rad/s]'])
        
        
    return data_out

In [ ]:
def initialize_print_progress(data_folders, notify_progress_threshold = 10):
    # initializes the parameters needed for the print_progress function
    
    number_folders = len(data_folders) -1       # function to show the progress of loading and saving the data
    current_folder_number = 0
    notify_progress_threshold_out = notify_progress_threshold            # threshhold to update the progress every 10%
        
    return current_folder_number, number_folders, notify_progress_threshold_out

In [ ]:
def print_progress(current_folder_number, number_folders, notify_progress_threshold):
    # prints the progress of loading and preparing the data every 10%

    current_folder_number += 1                  
    progress = 100 * current_folder_number/number_folders
    
    if progress >= notify_progress_threshold:
        
        if notify_progress_threshold == 10:
            
            print('')
            
        print(str(notify_progress_threshold) + '%')
        notify_progress_threshold += 10
        
    return current_folder_number, notify_progress_threshold

In [ ]:
def get_folder_list(input_dir = '', output_dir = '', save_seperatly = False, save_all = False):
    # gets a list of all folders in the input_dir and trys to create an outputfolder for the data is suppost
    # to be saveed. Either with the given output_dir, or the function creates a folder for the output calles:
    # raw_data_all/ for the output files.
    
    data_folders = glob.glob(input_dir + '*')   # gets a list of all folders in the input directory
    
    if save_all or save_seperatly:
    
        try:
            
            if output_dir:
                
                os.makedirs(output_dir)             # trying to make the passed output folder
                
                if save_all:
                    
                    os.makedirs(output_dir + 'raw_data_all/')
            
            else:
                
                output_dir = input_dir + 'raw_data/'
                
                if save_all:
                    
                    os.makedirs(output_dir + 'raw_data_all/')   
                    
                else:
                    
                    os.makedirs(output_dir) 
                    
                        # trying to make the automatic output folder
                
            print('\nCreation of the directory %s succesfull.' % output_dir)
            
        except OSError:
            
            print("\nCreation of the directory %s failed." % output_dir)
            
    return data_folders, output_dir

In [ ]:
def get_measurement_data(data_folder = ''):
    # loads the measurement data of the given subfolder which should contain the Accelerometer.csv and
    # Gyroscope.csv file. After both are loaded in, the time vector of both is checked to allign the data.
    # To achieve this the overhanging data is cut off and the data is allignd as close as possible.
    # Finally the data of one of the sensors is interpolated so that the time vectors match and the data can
    # be used reliably in the preprocessing. 
    # The output of this function is the trial information as DataFrames (gait, subject_number, trial) as well
    # as the joined and alligned sensor data as a DataFrame.

    folder_name = Path(data_folder).name
    
    empty_df = pd.DataFrame()
    
    if 'subject' not in folder_name:
        
        return empty_df, empty_df, empty_df, empty_df
    
    
    subject_number_str = folder_name[7:10]
    
    if not subject_number_str[2].isdigit():
            
            subject_number_str = subject_number_str[0:2]
    
    
    subject_number = pd.DataFrame(data = ([subject_number_str]), columns = ['subject_number'])
    
    trial_number = pd.DataFrame(data = ([folder_name[-2:]]), columns = ['trial_number'])
        
    data_accelerometer_dir = data_folder + '/Accelerometer.csv'
    data_gyroscope_dir = data_folder + '/Gyroscope.csv'
    
    try:
    
        data_accelerometer = pd.read_csv(data_accelerometer_dir)
        data_gyroscope = pd.read_csv(data_gyroscope_dir)  
    
    except Exception:
    
        print('There was an Error loading the Data: %s' % data_folder)
        
        return empty_df, empty_df, empty_df, empty_df
        
    data_accelerometer = unify_measurement_label(data_accelerometer, data_accelerometer_dir)
    data_gyroscope = unify_measurement_label(data_gyroscope, data_gyroscope_dir)

    if data_accelerometer.empty or data_gyroscope.empty:
        
        return empty_df, empty_df, empty_df, empty_df

    data_local = data_accelerometer.merge(data_gyroscope, left_on = 'Time [s]', right_on = 'Time [s]') # joins the data if the time vectors are aligned
    data_local = data_local.reset_index()
    data_local = data_local.drop(columns = 'index')
    
    if (not data_local.empty) and (not (data_local.iloc[0]['Time [s]'] == 0)):
        
        for i in range(data_local.iloc[:]['Time [s]'].size):
            
            data_local.at[i, 'Time [s]'] = data_local.iloc[i]['Time [s]'] - data_local.iloc[0]['Time [s]']
    
#------------------------------------------------------------------------------------------------------------------------------------#          
    
    if data_local.empty or ((data_local.size/data_local.columns.size) < (0.98 * data_accelerometer.size/data_accelerometer.columns.size)):

        # the following code trys to allign the data of the accelerometer and gyroscope if they are misaligned
        
        frequency_accelerometer = get_frequency(data_accelerometer).iloc[0]['frequency [Hz]']
        frequency_gyroscope = get_frequency(data_gyroscope).iloc[0]['frequency [Hz]']
        
        frequency_ratio = frequency_accelerometer/frequency_gyroscope
        
        offset_frequency = abs(1-frequency_ratio)     # computes the difference in freqeuncy

        if (abs(2 - offset_frequency) < 0.05):
            
            data_accelerometer = data_accelerometer[0::2]
            
        elif (abs(3 - frequency_ratio) < 0.05):
            
            data_accelerometer = data_accelerometer[0::3]
            
        elif (abs(6 - frequency_ratio) < 0.05):
            
            data_accelerometer = data_accelerometer[0::6]
            
        elif (abs(0.5 - frequency_ratio) < 0.05):
            
            data_gyroscope = data_gyroscope[0::2]
            
        elif (abs(1/3 - frequency_ratio) < 0.05):
            
            data_gyroscope = data_gyroscope[0::3]
            
        elif (abs(1/6 - frequency_ratio) < 0.05):
            
            data_gyroscope = data_gyroscope[0::6]
            
            
        frequency_accelerometer = get_frequency(data_accelerometer).iloc[0]['frequency [Hz]']
        frequency_gyroscope = get_frequency(data_gyroscope).iloc[0]['frequency [Hz]']
            
        frequency_ratio = frequency_accelerometer/frequency_gyroscope
    
        if (abs(1 - frequency_ratio) > 0.05):
                
            print('The frequency of the acceloremeter and the gyroscope data of %s do not allign.' % folder_name)
            
            return empty_df, empty_df, empty_df, empty_df                # skips this measurement cause the frequencys are different
        
        iteration = 0
        offset_time_min = 1
        offset_time_iteration = 0
        
        if (data_gyroscope.iloc[0]['Time [s]'] - data_accelerometer.iloc[0]['Time [s]']) > 0:
        
            acc_starts = True       # means that the accelerometer measurement started before the gyroscope
            
        else:
            
            acc_starts = False      # means that the gyroscope measurement started before the accelerometer
        
        for i in range(min(data_accelerometer.iloc[:]['Time [s]'].size, data_gyroscope.iloc[:]['Time [s]'].size)): 
            
        # This loop computes the datapoint of the first starting sensor which has the lowest time distance to the starting
        # datapoint of the sensor which is starting first
            
            if acc_starts:
            
                offset_time_iteration = abs(data_gyroscope.iloc[0]['Time [s]'] - data_accelerometer.iloc[i]['Time [s]'])
                
            else:
                
                offset_time_iteration = abs(data_accelerometer.iloc[0]['Time [s]'] - data_gyroscope.iloc[i]['Time [s]'])
            
            offset_time_min = min(offset_time_min, offset_time_iteration)
            
            if offset_time_min == offset_time_iteration:
                    
                iteration = i
                
            elif offset_time_min < offset_time_iteration:
                
                break
            
        offset_datapoints = data_accelerometer.iloc[:]['Time [s]'].size - data_gyroscope.iloc[:]['Time [s]'].size
        
        # obove the difference in datapoints of the accelerometer and gyroscope are computed 
        
        if offset_datapoints > 0:
            
            for i in range(iteration):  # this loop erases all measurements of the first starting sensor that are 
                                        # previous to the starting point of the later starting sensor
                
                if acc_starts: 
                
                    data_accelerometer = data_accelerometer.drop([i])
                    
                else:
                    
                    data_gyroscope = data_gyroscope.drop([i])                        
                    
            offset_datapoints_new = data_accelerometer.iloc[:]['Time [s]'].size - data_gyroscope.iloc[:]['Time [s]'].size
            
            data_accelerometer = data_accelerometer.reset_index()
            data_accelerometer = data_accelerometer.drop(columns = 'index')
            data_gyroscope = data_gyroscope.reset_index()
            data_gyroscope = data_gyroscope.drop(columns = 'index')    
            
            if not offset_datapoints_new == 0:
                
                for i in range(abs(offset_datapoints_new)):     # this loop erases all measurements of the later stopping
                                                                # sensor that are after the shorter recording sensor stops
                    
                    if offset_datapoints_new > 0:
                            
                        data_accelerometer = data_accelerometer.drop([data_accelerometer.iloc[:]['Time [s]'].size - 1])
                            
                    else:
                            
                        data_gyroscope = data_gyroscope.drop([data_gyroscope.iloc[:]['Time [s]'].size - 1])  


        
        data_accelerometer = data_accelerometer.reset_index()
        data_accelerometer = data_accelerometer.drop(columns = 'index')
        data_gyroscope = data_gyroscope.reset_index()
        data_gyroscope = data_gyroscope.drop(columns = 'index')    
        
        data_accelerometer_time = pd.DataFrame(data = np.array(data_accelerometer['Time [s]']), columns = ['Time [s]'])
        data_gyroscope_time = pd.DataFrame(data = np.array(data_gyroscope['Time [s]']), columns = ['Time [s]'])

        data_time_merged = data_gyroscope_time.append(data_accelerometer_time)
        data_time_merged = data_time_merged.reset_index()
        data_time_merged = data_time_merged.drop(columns = 'index')
    
        data_gyroscope = data_gyroscope.drop(columns = 'Time [s]')
        data_gyroscope = data_time_merged.join(data_gyroscope)
        data_gyroscope = data_gyroscope.drop_duplicates(subset = ['Time [s]'])
        data_gyroscope = data_gyroscope.sort_values(by = ['Time [s]'])
        data_gyroscope = data_gyroscope.interpolate(method='linear', axis = 0)
        
        
        data_local = data_accelerometer.merge(data_gyroscope, left_on = 'Time [s]', right_on = 'Time [s]')
        data_local = data_local.reset_index()
        data_local = data_local.drop(columns = 'index')
        
        for i in range(data_local.iloc[:]['Time [s]'].size):    
            
            data_local.at[i, 'Time [s]'] = data_local.iloc[i]['Time [s]'] - data_local.iloc[0]['Time [s]']

#------------------------------------------------------------------------------------------------------------------------------------#       
        
    if 'normal' in data_folder:             # gets the gait (normal or impaired) out of the folder name
            
        gait = pd.DataFrame(data=([0]), columns = ['gait'])
        
    else:
        
        gait = pd.DataFrame(data=([1]), columns = ['gait'])
        
    data_local = data_local.join(gait).join(subject_number).join(trial_number)
    
#------------------------------------------------------------------------------------------------------------------------------------#   
    
    return data_local, subject_number, trial_number, gait

In [ ]:
def get_join_anthropometric_data(data_local, data_folder, subject_number, anthropology_file_dir = ''):
  # This functino loads the anthropometric data file either from the anthropology_file_dir if one is defined,
  # or if not it trys to load the file from the parent folder of the input_dir.
  # After loading all the data the anthropometric information of the current subject is loaded and unified (
  # male, M, man -> 1, etc.). Then it is added to the sensor data DataFrame. This DataFrame is then returned
  # from the function.

    if not anthropology_file_dir:
    
        anthropology_file_dir = str(Path(data_folder).parent.parent) + '/anthro.xlsx'
        
    anthropometric_data_all = pd.read_excel(anthropology_file_dir)
    
    for subject_number_anthro in anthropometric_data_all.iloc[:]['subject no.']:

        if int(str(subject_number_anthro)[1:]) == int(subject_number.iloc[0]['subject_number']):
                
            anthropometric_data_list = anthropometric_data_all.iloc[int(str(subject_number_anthro)[1:])-1][:]
            anthropometric_data = pd.DataFrame(anthropometric_data_list)
            
            anthropometric_data = anthropometric_data.transpose()
            anthropometric_data = anthropometric_data.reset_index()
   
            anthropometric_data = anthropometric_data.drop(columns = 'subject no.').drop(columns = 'index')
            anthropometric_data = anthropometric_data.rename(columns = {'sex': 'subject_sex', 'age': 'subject_age [y]',
                                                                        'height [m]': 'subject_height [m]', 
                                                                        'weight [kg]': 'subject_weight [kg]'})
                                       
            if anthropometric_data.empty:
                
                return pd.DataFrame()
    
            anthropometric_data = unify_anthropometric_data(anthropometric_data)
            
            if not anthropometric_data.empty:

                data_local = data_local.join(anthropometric_data)   # adding the anthropology data to the rest of the dataframe
            
                break
            
            else: 
                
                return pd.DataFrame()
        
    return data_local

In [ ]:
def load_join_label(input_dir='', output_dir='', anthropology_file_dir = '', 
                    save_seperatly = False, save_all = False, return_data = True):
    
    if not input_dir:       # checking if the string is empty
        
        print('\nError: string of input directory is empty.')
        
        return
    
    if return_data:
        
        data_all = pd.DataFrame()
    
#------------------------------------------------------------------------------------------------------------------------------------#

    data_folders, output_dir = get_folder_list(input_dir, output_dir, save_seperatly, save_all)
        
    current_folder_number, number_folders, notify_progress_threshold = initialize_print_progress(data_folders, notify_progress_threshold = 10)
        
#------------------------------------------------------------------------------------------------------------------------------------#
    
    for data_folder in data_folders:                # loading the accelerometer and gyroscape data, joining and labeling it
        
        current_folder_number, notify_progress_threshold = print_progress(current_folder_number, number_folders, notify_progress_threshold)
        
        data_local, subject_number, trial_number, gait = get_measurement_data(data_folder)
        
        if data_local.empty:
            
            continue
        
        data_local = get_join_anthropometric_data(data_local, data_folder, subject_number, anthropology_file_dir)
        
        if data_local.empty:
            
            continue
        
        frequency = get_frequency(data_local)       # getting the frequency of the data
        data_local = data_local.join(frequency)     # adding the frequency to the rest of the dataframe
        
#------------------------------------------------------------------------------------------------------------------------------------#            
            
        if return_data:   # adding the current data to the matrix with all data
            
            if data_all.empty:
                
              data_all = data_local

            else:

              data_all = data_all.append(data_local, ignore_index = True, sort=False)
     
        if save_seperatly:          # saving the data as csv in the output directory
        
            data_local.to_csv(output_dir + 'raw_data_subject' + str(subject_number.iloc[0]['subject_number']) + '_gait' 
                              + str(data_local.iloc[0]['gait'])[0] + '_trial' + str(trial_number.iloc[0]['trial_number']) + '.csv')

#------------------------------------------------------------------------------------------------------------------------------------#         
        
    if save_seperatly or save_all:
        
        if save_all:
            
            data_all.to_csv(output_dir + 'raw_data_all/raw_data_all.csv')
        
        print('\nThe raw data was succesfully saved in: %s.' % output_dir)    
    
    if return_data:

        return data_all

    return 

In [ ]:
data_all_prepared = load_join_label(input_dir = input_dir, output_dir = output_dir, save_seperatly = True, save_all = False, return_data = True)

# 2. Data preprocessing

Input is a dataframe which was created in the first section.

In [ ]:
path_to_raw_data = '/content/drive/MyDrive/Colab Notebooks/CIE Data/raw_data/'
output_directory="../Smartphone2"

## Filter Noise

In [ ]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    # This function applies the Butterworth filter
    # Input: data: The data to be filtered
    #        fs: Sample frequency
    #        lowcut, highcut: Desired cutoff frequencies (in Hz)
    #        order: The order of the filter: Transmission loss of order*20 dB per frequency decade
    # Output: y: filtered data 
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype='band', output='sos')
    y = sosfilt(sos, data)
    return y

def butter_bandpass_DataFrame(data, lowcut_acc=1, lowcut_gyr=0.25, highcut_acc=20, highcut_gyr=30, order=4):
    # Applies the Butterworth filter for a whole DataFrame
    # Input: data: DataFrame to be filtered 
    #        lowcut_acc, lowcut_gyr, highcut_acc, highcut_gyr: Desired cutoff frequencies (in Hz) for accelerometer and gyroscope
    #        order: The order of the filter: Transmission loss of order*20 dB per frequency decade
    # Output: data: Filtered DataFrame
    col = data.columns
    fs = data.iloc[0]['frequency [Hz]']

    X_acc = data[col[2]].to_numpy()
    Y_acc = data[col[3]].to_numpy()
    Z_acc = data[col[4]].to_numpy()
    X_gyr = data[col[5]].to_numpy()
    Y_gyr = data[col[6]].to_numpy()
    Z_gyr = data[col[7]].to_numpy()
    
    X_acc_filtered = butter_bandpass_filter(X_acc, lowcut_acc, highcut_acc, fs, order)
    Y_acc_filtered = butter_bandpass_filter(Y_acc, lowcut_acc, highcut_acc, fs, order)
    Z_acc_filtered = butter_bandpass_filter(Z_acc, lowcut_acc, highcut_acc, fs, order)
    X_gyr_filtered = butter_bandpass_filter(X_gyr, lowcut_gyr, highcut_gyr, fs, order)
    Y_gyr_filtered = butter_bandpass_filter(Y_gyr, lowcut_gyr, highcut_gyr, fs, order)
    Z_gyr_filtered = butter_bandpass_filter(Z_gyr, lowcut_gyr, highcut_gyr, fs, order)
    
    data.loc[:]['acceleration_x [m/s^2]'] = X_acc_filtered
    data.loc[:]['acceleration_y [m/s^2]'] = Y_acc_filtered
    data.loc[:]['acceleration_z [m/s^2]'] = Z_acc_filtered
    data.loc[:]['angular_velocity_x [rad/s]'] = X_gyr_filtered
    data.loc[:]['angular_velocity_y [rad/s]'] = Y_gyr_filtered
    data.loc[:]['angular_velocity_z [rad/s]'] = Z_gyr_filtered

    return data

def savitzky_gola(data, window_length, polyorder):
    # This function applies the Savitzky_gola filter
    # Input: data: The data to be filtered
    #        window_length: The length of the filter window(the number of coefficients): Must be odd
    #        polyorder: The order of the polynomial used to fit the samples: Must be < window_length
    # Output: y: filtered data
    y = signal.savgol_filter(data, window_length, polyorder, mode = 'nearest', axis=0) #window length: odd positive integer, polyorder < window length
    return y

## Extract Motion Sequence

In [ ]:
def remove_beginning_and_end(data, standard_deviation_threshold = 1):
    #   This function automatically detects the start and end of the walking area and removes the non-relevant data before the start and after the end
    #   Input:      data: Data in form of a pd.DataFrame and a 
    #               standard_deviation_threshold: Threshold for the standard deviation at which the data are to be considered malicious
    #   Output:     data: Data with removed beginning and end
    try:
        print('\nData set: gait '+ str(data.iloc[0]['gait'])+ ', subject_number ' + str(data.iloc[0]['subject_number']) + ', trial_number ' + str(data.iloc[0]['trial_number']))
        
        # Import acceleration in z direction and time vector
        col = data.columns
        T_acc = data[col[1]].to_numpy()
        Z_acc = data[col[4]].to_numpy()
        
        # Remove the offset of the z acceleration
        offset = np.mean(Z_acc)
        Z_acc = Z_acc - offset

        # Sample rate and desired cutoff frequencies (in Hz) for Butterworth filter.
        fs = data.iloc[0]['frequency [Hz]']
        lowcut = 0.5
        highcut = 3
        
        # Apply Butterworth filter 
        Z_acc_filter = butter_bandpass_filter(Z_acc, lowcut, highcut, fs, order=5)

        # Assumed interval of one step in seconds. Assumed to be 1 second. If 1s proves to be insufficient, it can be adjusted later.
        interv = 1

        # Approximating number of timesteps for one interval
        intlength = int(interv*fs)

        # Find peaks for beginning and end detection
        peaks, _ = signal.find_peaks (Z_acc_filter, distance = intlength)
        peaks_unfiltered, _ = signal.find_peaks (Z_acc, distance = intlength)
        
        # Arrays to find beginning and end
        zmax = Z_acc_filter[peaks] # List of all amplitudes of the peaks found in the filtered data
        middle = int(len(Z_acc_filter)/2) # Middle of the measuring points
        zmax_middle = abs(np.array(peaks)-middle).argmin() # Corresponding middle in zmax
        middle_area = zmax[zmax_middle - 7 : zmax_middle + 8] # Array of zmax for an area in the middle: In this area the person walks in any case                                              
        standard_deviation = np.std(middle_area) # The average amplitude of the middle area
        average_zmax = np.mean(middle_area) # The standard deviation of the amplitude of the middle section                                              
        divergent = [zmax[i] > average_zmax - standard_deviation * 4 and zmax[i] < average_zmax + standard_deviation * 4 for i in range(len(zmax))] # Does the amplitude of the peak of the filtered data deviate from the mean value of the middle area only within static limits
        zmax_unfiltered = Z_acc[peaks_unfiltered] # List of all amplitudes of the peaks found in the  unfiltered data
        average_zmax_unfiltered = np.mean(zmax_unfiltered) 
        mean_unfiltered = np.mean(Z_acc[middle - 7 * intlength : middle + 7 * intlength]) # The mean value of the unfiltered data for an area in the middle 
        
        # Definition of limits that are later used to detect peaks with small aplitude
        if average_zmax > 3:
            limit = 1.2
        else:
            limit = 0.69
            
        # Definition of the time threshold
        time_threshold = int(intlength/2)

        # Decide whether the standard deviation is too large compared to the specified standard deviation threshold
        if standard_deviation > standard_deviation_threshold:
            print('\nThe data set: gait '+ str(data.iloc[0]['gait'])+ ', subject_number ' + str(data.iloc[0]['subject_number']) + ', trial_number ' + str(data.iloc[0]['trial_number']) + ' is removed.')
            data = pd.DataFrame()
        else:  
            
            ## Find beginning

            # Find large amplitude changes of the peaks of the filtered signal compared to a region in the middle
            j = zmax_middle
            while j > 0:
                if divergent[j] == False: 
                    beginning = peaks[j+1]
                    break
                else:
                    j -= 1     
            if j == 0 and (Z_acc[peaks_unfiltered[0]] > 7.5 or Z_acc[peaks_unfiltered[1]] > 7.5):
                j = 1       
            beginning = peaks[j]

            # Find peaks of the filtered signal with really small aplitude close to zero
            for i in range(zmax_middle):
                if zmax[i] > -limit and zmax[i] < limit and beginning < peaks[i+1]:
                    beginning = peaks[i+1] 
            
            # Find areas where the amplitude is always below an amplitude limit for a given time or areas where the local average is much higher or lower than the average for an area in the middle
            area_zero = False
            l = int(len(Z_acc)/2)
            if beginning > 2*time_threshold:
                start = beginning + time_threshold
            else:
                start = beginning - time_threshold
            while l > start:
                if len(Z_acc[int(l-time_threshold):l]) == time_threshold:
                    mean = np.mean(Z_acc[int(l-time_threshold):l])
                else: 
                    mean = np.mean(Z_acc[0:l])
                if mean > average_zmax* 1.3 + mean_unfiltered or mean < -average_zmax*1.3 + mean_unfiltered:
                    difference = True
                    l = int(l+4*time_threshold)
                else:
                    difference = False
                for m in range(int(l-time_threshold),l):
                    if Z_acc[m] > -average_zmax_unfiltered/6 and Z_acc[m] < average_zmax_unfiltered/6 or difference == True:
                        area_zero = True
                        continue
                    else:
                        area_zero = False
                        break
                if area_zero == False:
                    l = int(l-time_threshold)
                else:
                    next_peak = abs(np.array(peaks)-l).argmin()
                    if peaks[next_peak] < l:
                        beginning = peaks[next_peak+1]
                    else:
                        beginning = peaks[next_peak]
                    break
                   
            ## Find ending

            # Find large amplitude changes of the peaks of the filtered signal compared to a region in the middle
            k = zmax_middle
            while k < len(divergent) - 1:
                if divergent[k] == False:
                    ending = peaks[k-1]
                    break
                else:
                    k += 1     
            if k == len(divergent) - 1 and (Z_acc[peaks_unfiltered[len(peaks_unfiltered)-1]] > 7.5 or Z_acc[peaks_unfiltered[len(peaks_unfiltered)-2]] > 7.5):
                k = len(divergent) - 2    
            ending = peaks[k]
            
            # Find peaks of the filtered signal with really small aplitude close to zero
            for i in range(zmax_middle, len(zmax)-1):
                if zmax[i] > -limit and zmax[i] < limit and ending > peaks[i-1]:
                    ending = peaks[i-1]
                    break

            # Find areas where the amplitude is always below an amplitude limit for a given time or areas where the local average is much higher or lower than the average for an area in the middle
            area_zero = False     
            l = int(len(Z_acc)/2)
            if ending < len(Z_acc) - 2*time_threshold:
                end = ending + time_threshold
            else:
                end = ending - time_threshold
            while l < end:
                if len(Z_acc[l:int(l+time_threshold)]) == time_threshold:
                    mean = np.mean(Z_acc[l:int(l+time_threshold)])
                else: 
                    mean = np.mean(Z_acc[l:len(Z_acc)-1])
                if mean > average_zmax*1.3+mean_unfiltered or mean < -average_zmax*1.3+ mean_unfiltered:
                    difference = True
                    l = int(l-4*time_threshold)
                else:
                    difference = False
                for m in range(l,int(l+time_threshold)):
                    if Z_acc[m] > -average_zmax_unfiltered/6 and Z_acc[m] < average_zmax_unfiltered/6 or difference == True:
                        area_zero = True
                        continue
                    else:
                        area_zero = False
                        break
                if area_zero == False:
                    l = int(l+time_threshold)
                else:
                    next_peak = abs(np.array(peaks)-l).argmin()
                    if peaks[next_peak] < l:
                        ending = peaks[next_peak]
                    else:
                        ending = peaks[next_peak-1]
                    break
            
            # If the found beginning is also the found end, the data set is removed
            if beginning==ending:
                print('\nThe data set: gait '+ str(data.iloc[0]['gait'])+ ', subject_number ' + str(data.iloc[0]['subject_number']) + ', trial_number ' + str(data.iloc[0]['trial_number']) + ' is removed.')
                data = pd.DataFrame()
            
            # Remove all data up to the found beginning and from the found end and save the information from the first line into the new first line
            else:                
                gait = data.loc[0,('gait')]
                subject_number = data.loc[0,('subject_number')]
                trial_number = data.loc[0,('trial_number')]
                subject_sex = data.loc[0,('subject_sex')]
                subject_age = data.loc[0,('subject_age [y]')]
                subject_height = data.loc[0,('subject_height [m]')]
                subject_weight = data.loc[0,('subject_weight [kg]')]
                frequency = data.loc[0,('frequency [Hz]')]  
                
                for i in range(beginning):
                    data= data.drop([i])
                for i in range(ending+1,len(Z_acc)):
                    data = data.drop([i]) 
                
                data.loc[beginning,('gait')]= gait
                data.loc[beginning,('subject_number')] = subject_number
                data.loc[beginning,('trial_number')] =  trial_number
                data.loc[beginning,('subject_sex')] = subject_sex
                data.loc[beginning,('subject_age [y]')] = subject_age
                data.loc[beginning,('subject_height [m]')] = subject_height
                data.loc[beginning,('subject_weight [kg]')] = subject_weight
                data.loc[beginning,('frequency [Hz]')] = frequency
                
        
    except Exception:
            
        print('\nFailed to remove the beginning and end of the data set: gait ' + str(data.iloc[0]['gait'])+ ', subject_number ' + str(data.iloc[0]['subject_number']) + ', trial_number ' + str(data.iloc[0]['trial_number']))
        
    return data

## Rotate Data

In [ ]:
def rotate_Data(data):
    #   This function calculates the gravity vector through the mean of the 
    #   acceleration data. It then performs a Principle component analysis in 
    #   order to rotate the g-vector into the x-axis. To eliminate the last DOF
    #   the data will be rotated around the x-axis to maximize the angular 
    #   velocity. (Criterion for max angular velocity is of emperical nature. It 
    #   seems to obtain the most uniform results.)
    #
    #   Input:      data: pd.DataFrame    unrotated Data
    #   Output:     data: pd.DataFrame    rotated Data
 

    # from DataFame to array
    col = data.columns
    acc = data[[col[2], col[3], col[4]]].to_numpy()
    gyro = data[[col[5], col[6], col[7]]].to_numpy()

    # get mean of accaleration components (should be gravitational accerlation as a vector)
    accmean = np.mean(acc, axis=0)

    # perform a singular value decomposition on g vector to get rotation matrix 
    M = np.outer(accmean.T, accmean)
    u, s, vh = np.linalg.svd(M)

    # rotate all the Data by rotation matrix from previous step
    acc = np.matmul(acc, u)
    gyro = np.matmul(gyro, u)

    # calculate updated g vector. should only now be in x-direction
    accmean1 = np.mean(acc, axis=0)

    # if g vector is in negative x-direction flip coordinate system 180 degrees around z axis, so it alwas points in positive x-direction
    if accmean1[0] < 0:
        r = R.from_euler('z', 180, degrees=True).as_matrix()
        acc = np.matmul(acc, r)
        gyro = np.matmul(gyro, r)

    # After pca of g-vector, there is still 1 degree of freedom (rotation around new x-axis)
    # rotate around x-axis so that angular velocity around z-axis gets maximized
    maxy = []
    gyro1 = gyro
    #rotate to find maximum value
    for i in range(0,360):
        r = R.from_euler('x', 1, degrees=True).as_matrix()
        gyro1 = np.matmul(gyro1, r)
        maxy.append(gyro1[:, 2].max())

    angle = max(maxy)
    angle = maxy.index(angle)

    # Rotate around found angle
    r = R.from_euler('x', angle, degrees=True).as_matrix()
    acc = np.matmul(acc, r)
    gyro = np.matmul(gyro, r)

    # write updated data back into DataFrame
    data[[col[2], col[3], col[4]]] = acc
    data[[col[5], col[6], col[7]]] = gyro

    return data

## Segment Trial into Samples

In [ ]:
def getsteps(data, plotme=False):
    # This function tries to detect the starting positions of each step.
    # This function defines the starting point at the peaks of the acceleration 
    # data in x-direction. x-Direction is chosen since rotation yields the 
    # highest confidence and therefore regularity in this direction.
    #
    # input:  data:     pd.DataFrame with gait data.
    #         plotme:   boolean; Creates a plot if set to True. False by default
    # output: stepidx:  Array with indices of the starting points of the steps

    col = data.columns

    # Data to arrays
    t = data[col[1]].to_numpy()
    x = data[col[2]].to_numpy()

    # Filtering the data seems to improve the consitency in finding peaks, yet 
    # it does not change the location of them.
    # Sample rate and desired cutoff frequencies (in Hz) for Butterworth filter.
    fs = data.iloc[0]['frequency [Hz]']
    lowcut = 0.5
    highcut = 4

    # Apply Butterworth filter
    x = butter_bandpass_filter(x, lowcut, highcut, fs, order=5)

    # Assumed duration of one step in seconds.
    interv = 1

    # Approximating number of timesteps for one interval
    intlength = (np.abs(t - (interv+t[1]))).argmin()-1

    # Initializing output array
    stepidx = []
    diff = []

    # plot for visualization
    if plotme:
        plt.figure()
        plt.plot(t, x)

    # Defining start end endpoint of first search interval
    int_start = 0
    int_end = int(round(int_start + intlength))

    i = 0

    # Loop until the end. (i<100 as safty feature for while-loop)
    while int_end < len(x) and i < 100:

        # find maximum in interval
        stepidx.append(int_start + x[int_start: int_end].argmax())
        diff.append(stepidx[i]-stepidx[i-1])

        # update interval
        int_start = int(round(stepidx[i] + round(0.5*intlength)))
        int_end = int(round(int_start + round(1*intlength)))

        i += 1

    # Evaluate quality of result.
    diff[0] = stepidx[0]
    diffstd = np.std(diff[1:-1])
    diffmed = int(np.median(diff))

    # If standard deviation between step lengths is too high, run once more with 
    # median steplength as better step length approximation. Interval where 
    # algorithm is looking for peak can now be narrower.
    if diffstd > 0.1*diffmed:

        # Updated search parameters
        intlength = diffmed
        int_start = int(round(x.argmax() % intlength))
        int_end = int(round(int_start + intlength))

        stepidx2 = []
        diff2 = []
        
        i = 0

        # Loop until the end. (i<100 as safty feature for while-loop)
        while int_end < len(x) and i < 100:
            # find maximum in interval
            stepidx2.append(int_start + x[int_start:int_end].argmax())
            diff2.append(stepidx2[i] - stepidx2[i - 1])

            # update interval
            int_start = int(round(stepidx2[i] + (diffmed - 0.5 * diffstd)))
            int_end = int(round(int_start + (diffstd + 0.5 * diffstd)))

            i += 1

        # plot for visualization
        if plotme:
            plt.plot(t[stepidx2], x[stepidx2], 'x', c='g')

        return stepidx2

    if plotme:
        plt.plot(t[stepidx], x[stepidx], 'x', c='r')

    return stepidx

## Resampling and normalization

In [ ]:
def resample(data, stepidx, step_length): 
    
    # Function for resampling data to same frame number
    # input arguments: data (acquired after rotate_Data() function)
    #                  stepidx (array retured from getsteps() function)
    #                  step_length (number of required data points for each step)
    #                  output_dir (directory to store the resulting resampled data files)
    # output: All_steps (contains data for each step)
    

    col=data.columns
    data = data.fillna(method ='ffill') 
    size = len(stepidx)                 # to get number of steps
    number = step_length                # 100: fixed value for number of required data points for each step
    d = pd.DataFrame()
    dnew = pd.DataFrame()
    All_steps = pd.DataFrame()
    
    for i in range(size-1):
                                            
        d = pd.DataFrame(data).set_index(data[col[0]])[stepidx[i]:stepidx[i+1]].copy(deep=True)     # separating each step
        d = sp.signal.resample(d,number)                                                            # resampling
       
    
        # resampling converts dataframe to numpy array
        # next step converts the obtained numpy array to dataframe again        
        
        
        dnew = pd.DataFrame(data=d, columns = ['steps_data_points', 'Time [s]','acceleration_x [m/s^2]', 'acceleration_y [m/s^2]', 'acceleration_z [m/s^2]'
                                               , 'angular_velocity_x [rad/s]', 'angular_velocity_y [rad/s]', 'angular_velocity_z [rad/s]', 'gait',
                                               'subject_number', 'trial_number', 'subject_sex', 'subject_age [y]',
                                               'subject_height [m]', 'subject_weight [kg]', 'frequency [Hz]'])
        
        
        # Normalization of the sensors data onto [-1,1]
        
        cols_to_norm = ['acceleration_x [m/s^2]', 'acceleration_y [m/s^2]', 'acceleration_z [m/s^2]', 'angular_velocity_x [rad/s]', 'angular_velocity_y [rad/s]', 'angular_velocity_z [rad/s]']
        
        dnew[cols_to_norm] = dnew[cols_to_norm].apply(lambda x: 2*((x - x.min()) / (x.max() - x.min()))-1)

        # Normalization of the anthromopetric data onto [0,1]

        age_norm = ['subject_age [y]']
        height_norm = ['subject_height [m]']
        weight_norm = ['subject_weight [kg]']
        
        dnew[age_norm] = dnew[age_norm].apply(lambda x: ((x - 21) / (47 - 21)))
        dnew[height_norm] = dnew[height_norm].apply(lambda x: ((x - 1.55) / (2.0 - 1.55)))
        dnew[weight_norm] = dnew[weight_norm].apply(lambda x: ((x - 46) / (200 - 46)))

        del dnew['steps_data_points']

        dnew.insert(0, 'step_number', i+1)
        
        All_steps = All_steps.append(dnew, ignore_index = True)
    
        
    return All_steps

## Check and remove malicious sequences

In [ ]:
def initialize_plot_all_steps_of_one_trial(trial_information = ''): 
    # takes the information of the trial as input (subject number, gait, trial)
    # creates and returns a fig subplot for plotting the steps of one trial
        
    fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(20,10))
    fig.suptitle('Data of one trial\n' + trial_information, fontsize = 20)
    
    axs[0, 0].set_title('acceleration_x [m/s^2]')
    axs[0, 0].set_ylim(-1,1)

    axs[0, 1].set_title('acceleration_y [m/s^2]')
    axs[0, 1].set_ylim(-1,1)

    axs[0, 2].set_title('acceleration_z [m/s^2]')
    axs[0, 2].set_ylim(-1,1)

    axs[1, 0].set_title('angular_velocity_x [rad/s]')
    axs[1, 0].set_ylim(-1,1)

    axs[1, 1].set_title('angular_velocity_y [rad/s]')
    axs[1, 1].set_ylim(-1,1)

    axs[1, 2].set_title('angular_velocity_z [rad/s]')
    axs[1, 2].set_ylim(-1,1)
    
    return fig, axs

In [ ]:
def plot_all_steps_of_one_trial(fig, axs, malicious = False, data = pd.DataFrame()):
  # This function adds the input step data to the existing subplots and plots it
  # red if the step was found to be malicious or blue if it was found to be fine.
    
    data = data.reset_index()
    
    plt.ioff()

    if malicious:
        
        axs[0, 0].plot(data['acceleration_x [m/s^2]'], c = 'red')
        axs[0, 1].plot(data['acceleration_y [m/s^2]'], c = 'red')
        axs[0, 2].plot(data['acceleration_z [m/s^2]'], c = 'red')
        axs[1, 0].plot(data['angular_velocity_x [rad/s]'], c = 'red')
        axs[1, 1].plot(data['angular_velocity_y [rad/s]'], c = 'red')
        axs[1, 2].plot(data['angular_velocity_z [rad/s]'], c = 'red')
    
    else:
        
        axs[0, 0].plot(data['acceleration_x [m/s^2]'], c = 'blue')
        axs[0, 1].plot(data['acceleration_y [m/s^2]'], c = 'blue')
        axs[0, 2].plot(data['acceleration_z [m/s^2]'], c = 'blue')
        axs[1, 0].plot(data['angular_velocity_x [rad/s]'], c = 'blue')
        axs[1, 1].plot(data['angular_velocity_y [rad/s]'], c = 'blue')
        axs[1, 2].plot(data['angular_velocity_z [rad/s]'], c = 'blue')
    
    return fig, axs

In [ ]:
def remove_malicious_sequences_per_trial(data = pd.DataFrame(), offset_threshold = 0.3, quota_above_threshold_threshold = 0.4,
                               difference_std_indivually_threshold = 0.3, difference_std_mean_threshold = 0.2,
                               step_length = 100, show_plots = False):
  # This funciton takes the data of one trial as the input. First the mean function of all sensors of this trial as well as the average
  # standard deviation of the trial are computed. Then all steps of this trial are examinend by checking for 2 things:
  # 1.  The data of the step is distant (>'offset_threshold') to the mean data of this trial (sensorwise) for at least x % of the
  #     Datapoints of this trial (quota_above_threshold_threshold).
  # 2.  The std of this step is different to the mean std of this trial (both as mean of all sensors (difference_std_mean_threshold) and 
  #     for all sensors seperatly (difference_std_indivually_threshold))
  # If one of these thresholds is crossed, the step gets deleted out of the trial data.
  # The function returns the data of the trial which only contains the steps which were found to be normal.

    
    data = data.reset_index()
    data = data.drop(columns='index')
    
    data_out = data
    
    data_only_sensors = data[['acceleration_x [m/s^2]', 'acceleration_y [m/s^2]', 'acceleration_z [m/s^2]',
                                    'angular_velocity_x [rad/s]', 'angular_velocity_y [rad/s]', 'angular_velocity_z [rad/s]']]

    number_all_steps = int(data_only_sensors.size/(6*step_length))
    

    subject_number = round(data.iloc[0]['subject_number'])
    gait = round(data.iloc[0]['gait'])
    trial_number = round(data.iloc[0]['trial_number'])

    
    for i in range(number_all_steps):
        
        # getting the information of the current step

        current_position = i * step_length

        data_step = data.iloc[current_position: current_position+step_length]  

        if i == 0:
            
            # initializing the values for the first step overall
            
            data_trial_accelerometer_x = pd.DataFrame()
            data_trial_accelerometer_y = pd.DataFrame()
            data_trial_accelerometer_z = pd.DataFrame()
            data_trial_gyroscope_x = pd.DataFrame()
            data_trial_gyroscope_y = pd.DataFrame()
            data_trial_gyroscope_z = pd.DataFrame()


            # now that a new trial is reached the data of the previous trial is examined
            # therefore first the mean function of the sensors and the standard deviation are computed


            
        data_trial_accelerometer_x[str(i)] = np.array(data_step.iloc[:]['acceleration_x [m/s^2]'])
        data_trial_accelerometer_y[str(i)] = np.array(data_step.iloc[:]['acceleration_y [m/s^2]'])
        data_trial_accelerometer_z[str(i)] = np.array(data_step.iloc[:]['acceleration_z [m/s^2]'])
        data_trial_gyroscope_x[str(i)] = np.array(data_step.iloc[:]['angular_velocity_x [rad/s]'])
        data_trial_gyroscope_y[str(i)] = np.array(data_step.iloc[:]['angular_velocity_y [rad/s]'])
        data_trial_gyroscope_z[str(i)] = np.array(data_step.iloc[:]['angular_velocity_z [rad/s]'])
            
            
    data_trial_mean_accelerometer_x = data_trial_accelerometer_x.mean(axis = 1)
    data_trial_mean_accelerometer_y = data_trial_accelerometer_y.mean(axis = 1)
    data_trial_mean_accelerometer_z = data_trial_accelerometer_z.mean(axis = 1)
    data_trial_mean_gyroscope_x = data_trial_gyroscope_x.mean(axis = 1)
    data_trial_mean_gyroscope_y = data_trial_gyroscope_y.mean(axis = 1)
    data_trial_mean_gyroscope_z = data_trial_gyroscope_z.mean(axis = 1)


    data_trial_means_dict = {'acceleration_x [m/s^2]': data_trial_mean_accelerometer_x, 'acceleration_y [m/s^2]': data_trial_mean_accelerometer_y,
                              'acceleration_z [m/s^2]': data_trial_mean_accelerometer_z, 'angular_velocity_x [rad/s]': data_trial_mean_gyroscope_x,
                              'angular_velocity_y [rad/s]': data_trial_mean_gyroscope_y, 'angular_velocity_z [rad/s]': data_trial_mean_gyroscope_z}

    data_trial_means = pd.DataFrame(data = data_trial_means_dict)


    data_trial_std_obj = data_only_sensors.std(axis = 0)
    data_trial_std = pd.DataFrame(data_trial_std_obj).T
            
#------------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------------------#            
    
    for i in range(number_all_steps):
        
        current_position = i * step_length
        
        data_step = data.iloc[current_position: current_position+step_length]
        data_step = data_step.reset_index()
        data_step = data_step.drop(columns = 'index')
        data_step_only_sensors = data_only_sensors[current_position: current_position + step_length]
        data_step_only_sensors = data_step_only_sensors.reset_index()
        data_step_only_sensors = data_step_only_sensors.drop(columns = 'index')
        
        data_step_std_obj = data_step_only_sensors.std(axis = 0)
        data_step_std = pd.DataFrame(data_step_std_obj).T
    
        difference_std = (data_step_std - data_trial_std).abs()
        difference_std_mean = float(difference_std.mean(axis = 1))
    
        axis_list = data_step_only_sensors.columns
        
#------------------------------------------------------------------------------------------------------------------------------------#
    
        if ((difference_std > difference_std_indivually_threshold).any().any()) or (
            difference_std_mean > difference_std_mean_threshold):
    
            # checking if the std exceeds the allowed threshold and if so removing the data of that step
    
            malicious_std = True
        
            
            step_str = str(round(data_step.iloc[0]['step_number']))
    
            print_str = 'subject_number' + str(subject_number) + '_gait' + str(gait) + '_trial' + str(trial_number) + '_step' + step_str
    
            print('The data of %s was found to be malicious and will therefore be removed from the dataset.' % print_str
                 + ' The error was a transgression of the standard-deviation-threshold.')  
            
            
            try:
                data_out = data_out.drop(list(range(current_position, current_position + step_length)))
    
            except Exception:
                print('There was an error removing the data')                            
            
    
            
        else:
            malicious_std = False
            
#------------------------------------------------------------------------------------------------------------------------------------#

        above_threshold_counter = 0

        for j in range(step_length): 
        
        # checking for malicious steps

            for axis in axis_list:
            
            # comparing the data of the current step to the mean of the current trial

                data_iteration = float(data_step.iloc[j][axis])

                if abs(data_iteration - float(data_trial_means.at[j, axis])) > offset_threshold:
                
                # checking if the difference between the current value and the mean value exceed the threshold

                    above_threshold_counter += 1

#------------------------------------------------------------------------------------------------------------------------------------#

        if (above_threshold_counter/(step_length * 6)) > quota_above_threshold_threshold:
                
            # checking if the difference between the current value and the mean value exceeded the threshold...
            # ... more often than allowed by the quota_above_threshold_threshold (e.g. 40% of the datapoints)...
            # ... and if so deleting the data of this step                        
            
            malicious_offset = True
            

            step_str = str(round(data_step.iloc[0]['step_number']))

            print_str = 'subject_number' + str(subject_number) + '_gait' + str(gait) + '_trial' + str(trial_number) + '_step' + step_str
            
            print('The data of %s was found to be malicious and will therefore be removed from the dataset.' % print_str
                 + ' The error was a transgression of the quota_above_threshold_threshold.')
                                            
            
            try:

                data_out = data_out.drop(list(range(current_position, current_position + step_length)))
         
            except Exception:

                print('There was an error removing the data')

            
        else:
            malicious_offset = False
        
#------------------------------------------------------------------------------------------------------------------------------------#
  
        if show_plots:

            if i == 0:


                trial_information_str = 'subject_number' + str(subject_number) + '_gait' + str(gait) + '_trial' + str(trial_number)

                fig, axs = initialize_plot_all_steps_of_one_trial(trial_information_str)
                


            if malicious_std or malicious_offset:
                malicious = True
            else:
                malicious = False

            fig, axs = plot_all_steps_of_one_trial(fig, axs, malicious, data_step)

#------------------------------------------------------------------------------------------------------------------------------------#
    
    data_out = data_out.reset_index()
    data_out = data_out.drop(columns = 'index')
    
    if show_plots:
        plt.show()
    
#------------------------------------------------------------------------------------------------------------------------------------#

    return data_out

## Apply preprocessing steps to data


In [ ]:
# Here all preprocessing steps are applied to the data after loading and preparing (done trialwise)
# The data of all trials is added to a long DataFrame which is then output and saved as the data after proprocessing
# This data is then later used in the preperation for the NN.

datalist = [f for f in listdir(path_to_raw_data) if isfile(join(path_to_raw_data, f))]

step_length = 200
data_all = pd.DataFrame()

for filename in datalist:

    # Load the raw Data
    data = pd.read_csv(path_to_raw_data + filename)
    col = data.columns

    # Extract motion sequence
    data = remove_beginning_and_end(data, standard_deviation_threshold=1)
    
    if data.empty:
        continue
    
    # Rotate Data
    data = rotate_Data(data)   
    
    # Get starting positions of each step
    stepidx = getsteps(data)
    
    # Filter Data
    data = butter_bandpass_DataFrame(data)

    # Resample Data of steps to same frame number
    data = resample(data = data, stepidx = stepidx, step_length = step_length)
    
    data = data.drop(columns = ['Time [s]', 'subject_sex', 'subject_age [y]','frequency [Hz]'])
    
    data = remove_malicious_sequences_per_trial(data, step_length = step_length, offset_threshold=0.4, quota_above_threshold_threshold=0.15, difference_std_indivually_threshold = 0.2, difference_std_mean_threshold = 0.15, show_plots = False)        
    
    if data.empty:
        continue
    
    if data_all.empty:
        data_all = data
    else: 
        data_all = data_all.append(data)

        
# Saving the data   
output_filename = output_directory + '/data_after_preprocessing.csv'

data_all.to_csv(output_filename)

# 3.   Neural Network

Input: All preprocesed data as a csv file

To include the height or weight in the training, put its prespective boolean to True.

The NN-architecture (number of hidden layers, size of each hidden layer) can be set up inside the 'train_NN' function.

In [ ]:
input_file = ''
with_height = False
with_weight = False
number_epochs = 100

In [ ]:
def prepare_shuffling(data = pd.DataFrame, step_length = 200, weight = False, height = False):
  # This function creates a list of arrays where one arrays contains all steps of one subject.
  # Depending on the input parameters (weight, height) the input data of the NN is adjusted.
  # This list of arrays as well as the amount of columns in the data is then returned from the function.
 
    
    if weight and height:
        number_columns = 10
        data = data[['gait', 'subject_number', 'acceleration_x [m/s^2]', 'acceleration_y [m/s^2]', 'acceleration_z [m/s^2]',
                   'angular_velocity_x [rad/s]', 'angular_velocity_y [rad/s]', 'angular_velocity_z [rad/s]',
                    'subject_height [m]', 'subject_weight [kg]']]
        
    if weight and not height:
        number_columns = 9
        data = data[['gait', 'subject_number', 'acceleration_x [m/s^2]', 'acceleration_y [m/s^2]', 'acceleration_z [m/s^2]',
                   'angular_velocity_x [rad/s]', 'angular_velocity_y [rad/s]', 'angular_velocity_z [rad/s]',
                    'subject_weight [kg]']]
        
    if height and not weight:
        number_columns = 9
        data = data[['gait', 'subject_number', 'acceleration_x [m/s^2]', 'acceleration_y [m/s^2]', 'acceleration_z [m/s^2]',
                   'angular_velocity_x [rad/s]', 'angular_velocity_y [rad/s]', 'angular_velocity_z [rad/s]',
                    'subject_height [m]']]
        
    else:
        number_columns = 8
        data = data[['gait', 'subject_number', 'acceleration_x [m/s^2]', 'acceleration_y [m/s^2]', 'acceleration_z [m/s^2]',
                   'angular_velocity_x [rad/s]', 'angular_velocity_y [rad/s]', 'angular_velocity_z [rad/s]']]
 
        
    # reshaping the data into a stepwise array (3D)
    data_xy = data.to_numpy()
    steps = int(np.size(data_xy,0)/step_length)
    data_xy = np.array(np.reshape(data_xy,(steps, step_length,number_columns)))
    number_of_steps = (steps - (steps%6))
    steps_per_part = int(number_of_steps/6)
    data_xy = data_xy[:number_of_steps]
 
 
    # empty arrays
    zeros_counting = np.zeros((140))
 
    # finding the amount of steps of each subject and storing them in zeros_counting
    for i in range(140):
        for j in range(number_of_steps):
 
            if int(round(data_xy[j,0,1])) == i:     
                zeros_counting[i] = zeros_counting[i] + 1     
 
    # creating a list object with numpy arrays of the size of each subject (size ~ number of steps of the subject) inside            
    zeros_list_subjectwise = []    
    for i in range(140):
        zeros_list_subjectwise.append(np.zeros((int(zeros_counting[i]),200,number_columns))) 
 
    # filling the arrays in the list with the corresponding subject data
    for i in range(140):
        k=0
        for j in range((steps - (steps%6))):
 
            if int(round(data_xy[j,0,1])) == i:
 
                zeros_list_subjectwise[int(round(data_xy[j,0,1]))][k,:,:] = data_xy[j,:,:]
                k+=1
    
    return zeros_list_subjectwise, steps_per_part, number_columns

In [ ]:
def shuffle_data(data, steps_per_part, number_columns):
  # This function first shuffles the previously created list of arrays, so that the data is shuffled subjectwise.
  # Afterwards the shuffled data is assigned into 6 groups of arrays(5 for the 5fold and 1 as test data). This way all steps
  # of all trials of one subject are always in the same group. Finally these 6 groups are all split into NN input (x) and 
  # desired NN output (y). These arrays are then returned from the funciton and used as the input of the train_NN funciton.

    # shuffling the list of arrays
    random.seed(45)
    random.shuffle(data)
    data_subjectwise_shuffled_list = data
    
    
    # converting the list back to a numpy array
    list_empty = []
    number_empty = 0
    for i in range(140):    
        if len(data_subjectwise_shuffled_list[i])==0:
            number_empty = number_empty + 1
            list_empty.append(i) 

    notempty = 140 - number_empty
    length = math.floor(notempty/(number_columns-2))
    found = 1
    for i in range(140):
        if i == 0:
            data_train1 = data_subjectwise_shuffled_list[0]
            data_train2 = data_subjectwise_shuffled_list[0]
            data_train3 = data_subjectwise_shuffled_list[0]
            data_train4 = data_subjectwise_shuffled_list[0]
            data_train5 = data_subjectwise_shuffled_list[0]
            data_test = data_subjectwise_shuffled_list[0]

        if i in list_empty:
            continue
        else:
            if found < length + 1:
                data_train1 = np.r_[data_train1, data_subjectwise_shuffled_list[i]]
            if found > length and found < 2*length + 1:
                data_train2 = np.r_[data_train2, data_subjectwise_shuffled_list[i]]
            if found > 2*length and found < 3*length + 1:
                data_train3 = np.r_[data_train3, data_subjectwise_shuffled_list[i]]
            if found > 3*length and found < 4*length + 1:
                data_train4 = np.r_[data_train4, data_subjectwise_shuffled_list[i]]
            if found > 4*length and found < 5*length + 1:
                data_train5 = np.r_[data_train5, data_subjectwise_shuffled_list[i]]
            if found > 5*length and found < 6*length + 1:
                data_test = np.r_[data_test, data_subjectwise_shuffled_list[i]]   
            found = found+1 
            
    x_train1 = data_train1[:,:,2:]
    x_train2 = data_train2[:,:,2:]
    x_train3 = data_train3[:,:,2:]
    x_train4 = data_train4[:,:,2:]
    x_train5 = data_train5[:,:,2:]
    x_test = data_test[:,:,2:]

    y_train1 = data_train1[:,0,0]
    y_train2 = data_train2[:,0,0]
    y_train3 = data_train3[:,0,0]
    y_train4 = data_train4[:,0,0]
    y_train5 = data_train5[:,0,0]
    y_test = data_test[:,0,0]
    

    return x_train1, x_train2, x_train3, x_train4, x_train5, x_test, y_train1, y_train2, y_train3, y_train4, y_train5, y_test

In [ ]:
def train_NN(x_train1, x_train2, x_train3, x_train4, x_train5, x_test, y_train1, y_train2, y_train3, y_train4, y_train5, y_test, 
             epochs = 50, step_length = 200, number_columns = 6):
  # This function sets up the 5fold training data into training and validation (each run of the loop). Additionaly some monitoring functions are added
  # to stop overfitting and enable early stopping as well as saving the model.
  # Afterwards in each loop iteration the model is trained with the set NN-architecture and the respective training and validation data.
  # After the 5fold has finished the NN is evaluated with the test data.

    filepath='weights.best.hdf5'
    checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max') # Callback to save the Keras model or model weights 
    earlystopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=1,mode='min') # Early stopping function to prevent overfitting
    callback = [checkpointer, earlystopper]

    for i in range(5):

        print('\nFold: %s\n' %(i+1))

        if i == 0:
            x_train = np.r_[x_train2, x_train3, x_train4, x_train5]
            y_train = np.r_[y_train2, y_train3, y_train4, y_train5]
            x_val = x_train1
            y_val = y_train1
        elif i == 1:
            x_train = np.r_[x_train1, x_train3, x_train4, x_train5]
            y_train = np.r_[y_train1, y_train3, y_train4, y_train5]
            x_val = x_train2
            y_val = y_train2
        elif i == 2:
            x_train = np.r_[x_train2, x_train1, x_train4, x_train5]
            y_train = np.r_[y_train2, y_train1, y_train4, y_train5]
            x_val = x_train3
            y_val = y_train3
        elif i == 3:
            x_train = np.r_[x_train2, x_train3, x_train1, x_train5]
            y_train = np.r_[y_train2, y_train3, y_train1, y_train5]
            x_val = x_train4
            y_val = y_train4
        else:
            x_train = np.r_[x_train2, x_train3, x_train4, x_train1]
            y_train = np.r_[y_train2, y_train3, y_train4, y_train1]
            x_val = x_train5
            y_val = y_train5


# adjust the NN here:            
        model=tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(step_length,number_columns-2)), 
#             tf.keras.layers.Dense(600, activation ='sigmoid', kernel_regularizer = tf.keras.regularizers.l2(0.0001)),
#             tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(200, activation ='sigmoid', kernel_regularizer = tf.keras.regularizers.l2(0.0001)), 
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(100, activation ='sigmoid', kernel_regularizer = tf.keras.regularizers.l2(0.0001)), 
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1,activation="sigmoid")
                                ])

        # Compile model
        model.compile(optimizer = "adam",
                 loss = 'binary_crossentropy',
                 metrics = ['accuracy'])


        # Fit the model
        nn = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_val, y_val), verbose = 1, callbacks = callback)

    print('Training done!\n\nTesting results:')
        
    model.evaluate(x_test, y_test, verbose = 1)
    
    return nn

In [ ]:
data = pd.read_csv(input_file)

In [ ]:
data_for_shuffling, steps_per_part, number_columns  = prepare_shuffling(data = data, height = with_height, weight = with_weight)

In [ ]:
x_train1, x_train2, x_train3, x_train4, x_train5, x_test, y_train1, y_train2, y_train3, y_train4, y_train5, y_test = shuffle_data(data_for_shuffling, steps_per_part, number_columns)

In [ ]:
NN = train_NN(x_train1, x_train2, x_train3, x_train4, x_train5, x_test, y_train1, y_train2, y_train3, y_train4, y_train5, y_test, 
              epochs = number_epochs, step_length = 200, number_columns = number_columns)